### Introducción
- Trabajo de la asignatura INFO343 …
- Breve descripción del desafío y los datos

### Desafío
- Descripción incluyendo autores, año, institución, paper en el que se describe, etc.
- https://arxiv.org/pdf/1807.09902

### Datos
- Cuantos datos hay
- Cuantas clases distintas hay
- Cuantos datos están verificados
- Como se estructuran (carpeta test y train)
- Ejemplo de 1 espectrograma de cada clase

### Solución
- Paper
- Autores
- año
- Score
- https://dcase.community/documents/challenge2018/technical_reports/DCASE2018_Baseline_87.pdf

### Código
- Como ejecutar
- Estructura del código
- Archivos auxiliares
	- Funciones de cada archivo
- Modelo
	- Arquitectura
	- Explicación
- Código del modelo
- Ejecutar

# Código

El código que implementa el artículo descrito anteriormente se encuentra en el siguiente repositorio:

[Repositorio en GitHub](https://github.com/qiuqiangkong/dcase2018_task2/tree/master)

Además, este código está incluido en la carpeta "Código de referencia" del repositorio utilizado para este trabajo.

La Figura X muestra la estructura del código, que se compone principalmente de las siguientes carpetas y archivos:

- **utils**: Contiene funciones auxiliares, variables de configuración y scripts para crear los datos necesarios para el entrenamiento y validación del modelo.
- **pytorch**: Incluye el código de los modelos descritos anteriormente y la función que ejecuta el entrenamiento y validación.
- **runme.sh**: Archivo encargado de ejecutar el código en secuencia.

<div style="text-align: center;">
    <img src="./Imagenes/Estructura.png" alt="Estructura del Código" height="600">
    <p><em>Figura X: Estructura del repositorio.</em></p>
</div>

Este repositorio también incluye instrucciones sobre cómo ejecutar el código. Sin embargo, en el presente trabajo, este código se ha readaptado para ejecutarse mediante un Jupyter Notebook y para que funcione con versiones más recientes de las librerías utilizadas. Por lo tanto, todos los archivos del código de referencia están descritos en este notebook, pero serán presentados según su distribución en los archivos del código de referencia.

## Librerías

Para ejecutar el código en este trabajo, se deben instalar las librerías descritas en el archivo `requirements.txt`. Estas se han actualizado del código original a las siguientes versiones:

- **matplotlib**: 2.2.2 -> 3.9.1
- **numpy**: 1.14.5 -> 1.26.4
- **h5py**: 2.8.0 -> 3.11.0
- **pytorch**: 0.4.0 -> 2.3.1
- **librosa**: 0.6.1 -> 0.10.2
- **scikit-learn**: 0.19.1 -> 1.5.1
- **soundfile**: 0.10.2 -> 0.12.1

A continuación se encuentran todas las importaciones necesarias para la ejecución del programa:

In [1]:
import os
import argparse
import sys
import soundfile
import numpy as np
import librosa
import h5py
import time
import random
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import logging

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

from sklearn import preprocessing
import pickle

Los archivos y funciones presentes en la carpeta `utils` son los siguientes:

### utilities.py

Implementa las siguientes funciones auxiliares:

- **create_folder**: Crea un directorio si no existe.
- **get_filename**: Devuelve el nombre de un archivo a partir de una ruta y sin la extensión. Sin embargo, no es utilizada para la ejecución de esta tarea.
- **create_logging**: Crea un directorio para logs, un archivo de log numerado, y configura la salida del log tanto en archivo como en consola. Sin embargo, no es utilizada para la ejecución de esta tarea.
- **read_audio**: Lee un archivo de audio y, opcionalmente, lo resamplea a una frecuencia objetivo (variable `target_fs`).
- **calculate_scalar**: Calcula la media y la desviación estándar de un arreglo.
- **scale**: Normaliza un arreglo usando la media y la desviación estándar proporcionadas.
- **inverse_scale**: Desnormaliza un arreglo utilizando la media y la desviación estándar proporcionadas. Sin embargo, no es utilizada para la ejecución de esta tarea.
- **repeat_seq**: Repite varias veces una lista hasta que tenga una longitud de `time_steps`. Luego, corta la lista repetida para que su longitud sea exactamente `time_steps` y la devuelve.
- **calculate_accuracy**: Calcula el Accuracy de la clasificación comparando las predicciones con las etiquetas reales.
- **print_class_wise_accuracy**: Imprime y retorna el Accuracy por clase.
- **plot_class_wise_accuracy**: Grafica el Accuracy por clase. Sin embargo, no es utilizada para la ejecución de esta tarea.
- **write_testing_data_submission_csv**: Escribe los resultados al probar el modelo en un archivo CSV.

In [2]:
def create_folder(fd):
    if not os.path.exists(fd):
        os.makedirs(fd)
   
def read_audio(path, target_fs=None):
    (audio, fs) = soundfile.read(path)
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    if target_fs is not None and fs != target_fs:
        audio = librosa.resample(audio, orig_sr=fs, target_sr=target_fs)
        fs = target_fs
    return audio, fs

def calculate_scalar(x):
    if x.ndim == 2:
        axis = 0
    elif x.ndim == 3:
        axis = (0, 1)
    mean = np.mean(x, axis=axis)
    std = np.std(x, axis=axis)
    return mean, std

def scale(x, mean, std):
    return (x - mean) / std

def repeat_seq(x, time_steps):
    repeat_num = time_steps // len(x) + 1
    repeat_x = np.tile(x, (repeat_num, 1))[0 : time_steps]
    return repeat_x
    
def calculate_accuracy(output, target):
    acc = np.sum(output == target) / float(len(target))
    return acc
    
def print_class_wise_accuracy(output, target):
    """Print class wise accuracy."""
    global labels
    global ix_to_lb
    correctness = np.zeros(len(labels), dtype=np.int32)
    total = np.zeros(len(labels), dtype=np.int32)
    for n in range(len(target)):
        total[target[n]] += 1
        if output[n] == target[n]:
            correctness[target[n]] += 1
    class_wise_accuracy = correctness.astype(np.float32) / total
    logging.info('{:<30}{}/{}\t{}'.format(
        'event labels', 'correct', 'total', 'accuracy'))
    for (n, label) in enumerate(labels):
        logging.info('{:<30}{}/{}\t\t{:.2f}'.format(
            label, correctness[n], total[n], class_wise_accuracy[n]))
    class_wise_accuracy = np.array(class_wise_accuracy)
    return class_wise_accuracy, correctness, total

def write_testing_data_submission_csv(submission_path, audio_names, 
                                      sorted_indices):
    global kmax
    global ix_to_lb
    global corrupted_files
    # Write result to submission csv
    f = open(submission_path, 'w')
    f.write('fname,label\n')
    for (n, audio_name) in enumerate(audio_names):
        f.write('{},'.format(audio_name))
        predicted_labels = [ix_to_lb[sorted_indices[n, k]] for k in range(kmax)]
        f.write(' '.join(predicted_labels)) 
        f.write('\n')
    for audio_name in corrupted_files:
        f.write('{},{}\n'.format(audio_name, 'Acoustic_guitar'))
    f.close()
    print("Write result to {}".format(submission_path))

### average_precision.py

Este archivo implementa funciones para evaluar la precisión del modelo:

- **apk**: Calcula la precisión promedio de las primeras k predicciones (por defecto, los primeros 10 elementos).

- **mapk**: Extiende la función `apk` a múltiples listas, calculando la precisión promedio de las primeras k predicciones para cada lista y luego promediando estos valores (por defecto, los primeros 10 elementos).

In [3]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

### data_generator.py

Este archivo contiene una clase `DataGenerator` que se utiliza para generar datos para entrenar y validar el modelo y la clase `TestDataGenerator` para generar datos de prueba. Las descripciones de los métodos de estas clases son las siguientes:

#### Clase `DataGenerator`
- **`__init__`**: Inicializa la clase con los parámetros dados, carga los datos desde un archivo HDF5 y calcula las estadísticas para la normalización.
- **`get_audio_indexes`**: Obtiene los índices de los audios de entrenamiento y validación a partir del archivo CSV de validación.
- **`calculate_training_data_scalar`**: Calcula la media y la desviación estándar de los datos de entrenamiento.
- **`calculate_patch_bgn_fin_y_tuples`**: Calcula las posiciones de inicio y fin de los segmentos de audio, de los datos de entrenamiento.
- **`get_patch_bgn_fin_y_tuples_for_an_audio`**: Calcula las posiciones de inicio y fin del segmentos de un audio.
- **`generate_train`**: Genera lotes de datos para el entrenamiento.
- **`get_batch_x_y`**: Obtiene los datos de entrada y las etiquetas para un lote (batch).
- **`generate_validate_slices`**: Genera lotes de datos para la validación del modelo.
- **`transform`**: Normaliza los datos.

#### Clase `TestDataGenerator`
- **`__init__`**: Inicializa la clase con los parámetros dados, carga los datos desde un archivo HDF5 y hereda los métodos de la clase `DataGenerator` (mediante *super(TestDataGenerator, self)*).
- **`generate_test_slices`**: Itera sobre los índices de los audios de prueba y genera lotes de datos normalizados junto con los nombres de los audios. 

In [4]:
class DataGenerator(object):
    
    def __init__(self, hdf5_path, batch_size, time_steps, 
        validation_csv=None, holdout_fold=None, seed=1234):
        """
        Inputs:
          hdf5_path: str, path of hdf5 data
          batch_size: int
          time_stes: int, number of frames of a logmel spectrogram patch
          validate_csv: string | None, if None then use all data for training
          holdout_fold: int
          seed: int, random seed
        """
        # Parameters
        self.batch_size = batch_size
        self.random_state = np.random.RandomState(seed)
        self.validate_random_state = np.random.RandomState(0)
        global labels
        self.labels = labels
        global lb_to_ix
        self.time_steps = time_steps
        self.hop_frames = self.time_steps // 2
        self.classes_num = len(self.labels)
        # Load data
        load_time = time.time()
        hf = h5py.File(hdf5_path, 'r')
        self.audio_names = np.array([s.decode() for s in hf['filename'][:]])
        self.x = hf['feature'][:]
        self.bgn_fin_indices = hf['bgn_fin_indices'][:]
        event_labels = hf['label'][:]
        self.y = np.array([lb_to_ix[s.decode()] for s in event_labels])
        self.manually_verifications = hf['manually_verification'][:]
        hf.close()
        logging.info('Loading data time: {:.3f} s'.format(
            time.time() - load_time))
        # Load validation
        if validation_csv:
            self.train_audio_indexes, self.validate_audio_indexes = \
                self.get_audio_indexes(validation_csv, holdout_fold)
        else:
            self.train_audio_indexes = np.arange(len(self.audio_names))
            self.validate_audio_indexes = np.array([])                  
        logging.info('Training audios number: {}'.format(
            len(self.train_audio_indexes)))
        logging.info('Validation audios number: {}'.format(
            len(self.validate_audio_indexes)))         
        # calculate scalar
        (self.mean, self.std) = self.calculate_training_data_scalar()
        # Get training patches
        self.train_patch_bgn_fin_y_tuples = \
            self.calculate_patch_bgn_fin_y_tuples(self.train_audio_indexes)
        logging.info('Training patches number: {}'.format(
            len(self.train_patch_bgn_fin_y_tuples)))
    
    def get_audio_indexes(self, validation_csv, holdout_fold):
        """Get train and audio indexes from validation csv. 
        """
        df = pd.read_csv(validation_csv, sep=',')
        df = pd.DataFrame(df)
        folds = df['fold']
        train_audio_indexes = np.where(folds != holdout_fold)[0]
        validate_audio_indexes = np.where(folds == holdout_fold)[0]
        return train_audio_indexes, validate_audio_indexes

    def calculate_training_data_scalar(self):
        """Concatenate all training data and calculate scalar. 
        """
        train_bgn_fin_indices = self.bgn_fin_indices[self.train_audio_indexes]
        train_x_concat = []
        for [bgn, fin] in train_bgn_fin_indices:
            train_x_concat.append(self.x[bgn : fin]) 
        train_x_concat = np.concatenate(train_x_concat, axis=0)
        (mean, std) = calculate_scalar(train_x_concat)
        return mean, std

    def calculate_patch_bgn_fin_y_tuples(self, audio_indexes):
        """Calculate (bgn, fin, y) tuples for selecting patches for training. 
        """
        bgn_fin_indices = self.bgn_fin_indices[audio_indexes]
        patch_bgn_fin_y_tuples = []
        for n in range(len(audio_indexes)):
            [bgn, fin] = bgn_fin_indices[n]
            y = self.y[audio_indexes[n]]
            patch_tuples_for_this_audio = \
                self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y)  
            patch_bgn_fin_y_tuples += patch_tuples_for_this_audio
        # Print class wise number of patches
        patches_per_class = np.zeros(self.classes_num, dtype=np.int32)
        for k in range(self.classes_num):
            patches_per_class[k] = np.sum(
                [tuple[2] == k for tuple in patch_bgn_fin_y_tuples])
        if False:
            for k in range(self.classes_num):
                logging.info('{:<30}{}'.format(
                    self.labels[k], patches_per_class[k]))
        return patch_bgn_fin_y_tuples

    def get_patch_bgn_fin_y_tuples_for_an_audio(self, bgn, fin, y):
        """Get (bgn, fin, y) tuples in an audio. 
        """
        
        if fin - bgn <= self.time_steps:
            patch_tuples_for_this_audio = [(bgn, fin, y)]
            
        else:
            bgns = np.arange(bgn, fin - self.time_steps, self.hop_frames)
            patch_tuples_for_this_audio = []
            
            for bgn in bgns:
                patch_tuples_for_this_audio.append(
                    (bgn, bgn + self.time_steps, y))
                
        return patch_tuples_for_this_audio
    

    def generate_train(self):
        batch_size = self.batch_size
        patch_bgn_fin_y_tuples = self.train_patch_bgn_fin_y_tuples.copy()
        time_steps = self.time_steps
        patches_num = len(patch_bgn_fin_y_tuples)
        self.random_state.shuffle(patch_bgn_fin_y_tuples)
        iteration = 0
        pointer = 0
        while True:
            # Reset pointer
            if pointer >= patches_num:
                pointer = 0
                self.random_state.shuffle(patch_bgn_fin_y_tuples)
            # Get batch indexes
            batch_patch_bgn_fin_y_tuples = patch_bgn_fin_y_tuples[
                pointer: pointer + batch_size]    
            pointer += batch_size
            iteration += 1
            (batch_x, batch_y) = self.get_batch_x_y(
                self.x, batch_patch_bgn_fin_y_tuples)
            # Transform data
            batch_x = self.transform(batch_x)
            yield batch_x, batch_y
        
    def get_batch_x_y(self, full_x, batch_patch_bgn_fin_y_tuples):
        """Get batch_x and batch_y, repeat is audio is short. 
        """
        batch_x = []
        batch_y = []
        for (bgn, fin, y) in batch_patch_bgn_fin_y_tuples:
            batch_y.append(y)
            if fin - bgn == self.time_steps:
                batch_x.append(full_x[bgn : fin])
            else:
                batch_x.append(repeat_seq(full_x[bgn : fin], self.time_steps))
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y

    def generate_validate_slices(self, data_type, manually_verified_only, 
                                 shuffle, max_audios_num=None):
        """Generate patches in an audio. 
        Args:
          data_type: 'train' | 'validate'
          manually_verified_only: bool
          shuffle: bool
          max_audios_num: int, set maximum audios to speed up validation
        """
        if data_type == 'train':
            audio_indexes = np.array(self.train_audio_indexes)
        elif data_type == 'validate':
            audio_indexes = np.array(self.validate_audio_indexes)
        else:
            raise Exception('Incorrect data_type!')
        if manually_verified_only:
            manually_verified_indexes = np.where(
                self.manually_verifications[audio_indexes]==1)[0]
            audio_indexes = audio_indexes[manually_verified_indexes]
        if shuffle:
            self.validate_random_state.shuffle(audio_indexes)
        for (n, audio_index) in enumerate(audio_indexes):
            if n == max_audios_num:
                break
            [bgn, fin] = self.bgn_fin_indices[audio_index]
            y = self.y[audio_index]
            audio_name = self.audio_names[audio_index]
            patch_tuples_for_this_audio = \
                self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y)
            (batch_x_for_an_audio, _) = self.get_batch_x_y(
                self.x, patch_tuples_for_this_audio)
            batch_x_for_an_audio = self.transform(batch_x_for_an_audio)
            yield batch_x_for_an_audio, y, audio_name
                
    def transform(self, x):
        """Transform data. 
        Args:
          x: (batch_x, seq_len, freq_bins) | (seq_len, freq_bins)
        Returns:
          Transformed data. 
        """

        return scale(x, self.mean, self.std)
            
            
class TestDataGenerator(DataGenerator):
    
    def __init__(self, dev_hdf5_path, test_hdf5_path, time_steps, 
                 test_hop_frames):
        """Test data generator. 
        Args:
          dev_hdf5_path: str, path of development hdf5 file
          test_hdf5_path: str, path of test hdf5 file
          time_stes: int, number of frames of a logmel spectrogram patch
          test_hop_frames: int
        """
        super(TestDataGenerator, self).__init__(
            hdf5_path=dev_hdf5_path, 
            batch_size=None, 
            time_steps=time_steps,
            validation_csv=None)
        self.test_hop_frames = test_hop_frames
        global corrupted_files
        self.corrupted_files = corrupted_files
        # Load test data
        load_time = time.time()
        hf = h5py.File(test_hdf5_path, 'r')
        self.test_audio_names = np.array([s.decode() for s in hf['filename'][:]])
        self.test_x = hf['feature'][:]
        self.test_bgn_fin_indices = hf['bgn_fin_indices'][:]
        hf.close()
        logging.info('Loading data time: {:.3f} s'.format(
            time.time() - load_time))
    
    def generate_test_slices(self):
        test_hop_frames = self.test_hop_frames
        global corrupted_files
        corrupted_files = corrupted_files
        audio_indexes = range(len(self.test_audio_names))
        for (n, audio_index) in enumerate(audio_indexes):
            [bgn, fin] = self.test_bgn_fin_indices[audio_index]
            audio_name = self.test_audio_names[audio_index]
            if fin > bgn:
                patch_tuples_for_this_audio = \
                    self.get_patch_bgn_fin_y_tuples_for_an_audio(bgn, fin, y=None)
                (batch_x_for_an_audio, _) = \
                    self.get_batch_x_y(self.test_x, patch_tuples_for_this_audio)
                batch_x_for_an_audio = self.transform(batch_x_for_an_audio)
                yield batch_x_for_an_audio, audio_name

### config.py

Este archivo define las variables necesarias para la ejecución del resto de funciones:

- **sample_rate**: Especifica la tasa de muestreo de la señal de audio entrante. En este caso, es de 32000 muestras por segundo
    -  - Referencia: [Librosa Mel Filters](https://librosa.org/doc/latest/generated/librosa.filters.mel.html)
- **window_size**: Número de componentes FFT (2048 en este caso).
  - Referencia: [Librosa Mel Filters](https://librosa.org/doc/latest/generated/librosa.filters.mel.html)
- **overlap**: Número de puntos que se superpondrán entre segmentos del espectrograma (1024 muestras).
  - Referencia: [SciPy Spectrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html)
- **mel_bins**: Número de bandas Mel a generar en el banco de filtros Mel. Esto genera una matriz de transformación lineal para proyectar los bins de FFT en bins de frecuencia Mel.
  - Referencia: [Librosa Mel Filters](https://librosa.org/doc/latest/generated/librosa.filters.mel.html)
- **labels**: Lista que contiene los nombres de las diferentes clases de sonidos.
- **lb_to_ix**: Diccionario que mapea cada etiqueta de `labels` a su índice correspondiente.
- **ix_to_lb**: Diccionario que mapea cada índice de `labels` a su etiqueta correspondiente.
- **corrupted_files**: Lista que contiene los nombres de archivos de audio que se consideran corruptos o problemáticos.

In [5]:
sample_rate = 32000

window_size = 2048
overlap = 1024

mel_bins = 64

kmax = 3

labels = ['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 
          'Burping_or_eructation', 'Bus', 'Cello', 'Chime', 'Clarinet', 
          'Computer_keyboard', 'Cough', 'Cowbell', 'Double_bass', 
          'Drawer_open_or_close', 'Electric_piano', 'Fart', 'Finger_snapping', 
          'Fireworks', 'Flute', 'Glockenspiel', 'Gong', 'Gunshot_or_gunfire', 
          'Harmonica', 'Hi-hat', 'Keys_jangling', 'Knock', 'Laughter', 'Meow', 
          'Microwave_oven', 'Oboe', 'Saxophone', 'Scissors', 'Shatter', 
          'Snare_drum', 'Squeak', 'Tambourine', 'Tearing', 'Telephone', 
          'Trumpet', 'Violin_or_fiddle', 'Writing']

lb_to_ix = {lb: i for i, lb in enumerate(labels)}

ix_to_lb = {i: lb for i, lb in enumerate(labels)}

corrupted_files = ['0b0427e2.wav', '6ea0099f.wav', 'b39975f5.wav']

### create_validation.py
Este archivo contiene una única función destinada a la validación del modelo:
- **create_validation_folds_**: Crea una estructura para realizar la validación cruzada, la cual se guarda en un archivo CSV llamado `validate_meta.csv`.

In [6]:
def create_validation_folds(dataset_dir, workspace):
    """Create validation file with folds and write out to validate_meta.csv
    """
    random_state = np.random.RandomState(1234)
    folds_num = 4
    # Paths
    csv_path = os.path.join(dataset_dir, 'train.csv')
    # Read csv
    df = pd.DataFrame(pd.read_csv(csv_path))
    indexes = np.arange(len(df))
    random_state.shuffle(indexes)
    audios_num = len(df)
    audios_num_per_fold = int(audios_num // folds_num)
    # Create folds
    folds = np.zeros(audios_num, dtype=np.int32)
    for n in range(audios_num):
        folds[indexes[n]] = (n % folds_num) + 1
    df_ex = df
    df_ex['fold'] = folds
    # Write out validation csv
    out_path = os.path.join(workspace, 'validate_meta.csv')
    df_ex.to_csv(out_path)
    print("Write out to {}".format(out_path))

Se ejecutará esta función especificando que los datos se encuentran en la ruta `./Data` (carpeta no incluida debido al gran tamaño).

In [7]:
dataset_dir = "./Data"
workspace = "./"
create_validation_folds(dataset_dir, workspace)

Write out to ./validate_meta.csv


### features.py

Este archivo contiene la clase `LogMelExtractor`, que se encarga de extraer las características de los audios.

- **\_\_init\_\_**: Inicializa el extractor de características con los parámetros proporcionados y calcula el banco de filtros Mel.
- **transform**: Transforma un fragmento de audio en características utilizando el logaritmo de la multiplicación del banco de filtros Mel con el espectrograma del mismo.
- **calculate_logmel**: Lee un archivo de audio, lo normaliza y extrae sus características mediante el método `transform`.
- **calculate_features**: Lee los audios de un archivo CSV, extrae las características para cada uno y escribe estas características junto con otros metadatos en un archivo HDF5. HDF son las siglas de Hierarchical Data Format (Formato de datos jerárquicos) y fue diseñado para guardar y recuperar datos de/hacia archivos grandes estructurados.
  - Referencia: [HDF5 Manual](https://www.scayle.es/manual/es/hpc/software-instalado/hdf5)

In [8]:
class LogMelExtractor():

    def __init__(self, sample_rate, window_size, overlap, mel_bins):
        self.window_size = window_size
        self.overlap = overlap
        self.ham_win = np.hamming(window_size)
        self.melW = librosa.filters.mel(sr=sample_rate, 
                                        n_fft=window_size, 
                                        n_mels=mel_bins, 
                                        fmin=50., 
                                        fmax=sample_rate // 2).T
    
    def transform(self, audio):
        ham_win = self.ham_win
        window_size = self.window_size
        overlap = self.overlap
        [f, t, x] = signal.spectral.spectrogram(
                        audio, 
                        window=ham_win,
                        nperseg=window_size, 
                        noverlap=overlap, 
                        detrend=False, 
                        return_onesided=True, 
                        mode='magnitude') 
        x = x.T
        x = np.dot(x, self.melW)
        x = np.log(x + 1e-8)
        x = x.astype(np.float32)
        return x

def calculate_logmel(audio_path, sample_rate, extractor):
    (audio, _) = read_audio(audio_path, target_fs=sample_rate)
    audio = audio / np.max(np.abs(audio))
    feature = extractor.transform(audio)
    return feature

def calculate_features(dataset_dir, workspace, data_type, mini_data):
    """Write features and infos of audios to a hdf5 file.
    """
    global sample_rate
    global window_size
    global overlap
    global mel_bins
    global corrupted_files
    # Paths
    if data_type == 'development':
        audio_dir = os.path.join(dataset_dir, 'audio_train')
        meta_csv = os.path.join(dataset_dir, 'train.csv')
    elif data_type == 'test':
        audio_dir = os.path.join(dataset_dir, 'audio_test')
        meta_csv = os.path.join(dataset_dir, 'sample_submission.csv')
    if mini_data:
        hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                                 'mini_{}.h5'.format(data_type))
    else:
        hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                                 '{}.h5'.format(data_type))
    create_folder(os.path.dirname(hdf5_path))
    # Load csv
    df = pd.DataFrame(pd.read_csv(meta_csv))
    audio_names = []
    if data_type == 'development':
        manually_verifications = []
        target_labels = []
    for row in df.iterrows():
        audio_name = row[1]['fname']
        audio_names.append(audio_name)
        if data_type == 'development':
            target_label = row[1]['label']
            manually_verification = row[1]['manually_verified']
            target_labels.append(target_label)
            manually_verifications.append(manually_verification)
    # Use partial data when set mini_data to True
    if mini_data:
        audios_num = 300
        random_state = np.random.RandomState(0)
        audio_indexes = np.arange(len(audio_names))
        random_state.shuffle(audio_indexes)
        audio_indexes = audio_indexes[0 : audios_num]
        audio_names = [audio_names[idx] for idx in audio_indexes]
        if data_type == 'development':
            target_labels = [target_labels[idx] for idx in audio_indexes]
            manually_verifications = [manually_verifications[idx] 
                                      for idx in audio_indexes]
        print("Number of audios: {}".format(len(audio_names)))
    # Feature extractor
    extractor = LogMelExtractor(sample_rate=sample_rate,
                                window_size=window_size,
                                overlap=overlap,
                                mel_bins=mel_bins)
    # Write out to h5 file
    hf = h5py.File(hdf5_path, 'w')
    hf.create_dataset(
        name='feature',
        shape=(0, mel_bins),
        maxshape=(None, mel_bins),
        dtype=np.float32)
    calculate_time = time.time()
    bgn_fin_indices = []
    # Extract feature for audios
    for (n, audio_name) in enumerate(audio_names):
        # Extract feature
        if audio_name in corrupted_files:
            feature = np.zeros((0, mel_bins))
        else:
            audio_path = os.path.join(audio_dir, audio_name)
            feature = calculate_logmel(audio_path, sample_rate, extractor)
        # Write feature to hdf5
        bgn_indice = hf['feature'].shape[0]
        fin_indice = bgn_indice + feature.shape[0]
        hf['feature'].resize((fin_indice, mel_bins))
        hf['feature'][bgn_indice: fin_indice] = feature
        bgn_fin_indices.append((bgn_indice, fin_indice))
    # Write infos to hdf5
    hf.create_dataset(name='filename', 
                      data=[s.encode() for s in audio_names], 
                      dtype='S32')
    hf.create_dataset(name='bgn_fin_indices',
                      data=bgn_fin_indices,
                      dtype=np.int32)
    if data_type == 'development':
        hf.create_dataset(name='label', 
                          data=[s.encode() for s in target_labels], 
                          dtype='S32')
        hf.create_dataset(name='manually_verification',
                          data=manually_verifications,
                          dtype=np.int32)
    hf.close()
    
    print("Write out hdf5 file to {}".format(hdf5_path))
    print("Time spent: {} s".format(time.time() - calculate_time))

Al ejecutar el código, se crea la carpeta `features` donde se almacenan los archivos HDF5 llamado `development.h5` y `test.h5`. Sin embargo, esta carpeta no se incluye en el repositorio, ya que estos archivos son de gran tamaño.

In [9]:
data_type = 'development'
mini_data = False
calculate_features(dataset_dir, workspace, data_type, mini_data)

C:\Users\gluar\AppData\Local\Temp\ipykernel_25704\540075642.py:17: DeprecationWarning: Please import `spectrogram` from the `scipy.signal` namespace; the `scipy.signal.spectral` namespace is deprecated and will be removed in SciPy 2.0.0.
  [f, t, x] = signal.spectral.spectrogram(


Write out hdf5 file to ./features\logmel\development.h5
Time spent: 392.7382380962372 s


In [10]:
data_type = 'test'
calculate_features(dataset_dir, workspace, data_type, mini_data)

C:\Users\gluar\AppData\Local\Temp\ipykernel_25704\540075642.py:17: DeprecationWarning: Please import `spectrogram` from the `scipy.signal` namespace; the `scipy.signal.spectral` namespace is deprecated and will be removed in SciPy 2.0.0.
  [f, t, x] = signal.spectral.spectrogram(


Write out hdf5 file to ./features\logmel\test.h5
Time spent: 388.41344356536865 s


Los archivos y funciones presentes en la carpeta `pytorch` son los siguientes:

### model_pytorch

